In [1]:
#imports
import pickle
import torch
import math
import numpy as np
from collections import Counter

In [2]:
#project specific imports
from modules.utils import *
from modules.build_dataset import *

In [3]:
biotype_grouping = {
            'protein_coding': 'protein_coding',
            'processed_transcript': 'processed_transcript',
            'miRNA': 'processed_transcript',
            'misc_RNA': 'processed_transcript',
            'unprocessed_pseudogene': 'pseudogene',
            'antisense': 'processed_transcript',
            'retained_intron': 'processed_transcript',
            'processed_pseudogene': 'pseudogene',
            'rRNA_pseudogene': 'pseudogene',
            'sense_intronic': 'processed_transcript',
            'lincRNA': 'processed_transcript',
            'snoRNA': 'processed_transcript',
            'snRNA': 'processed_transcript',
            'transcribed_unprocessed_pseudogene': 'pseudogene',
            'translated_processed_pseudogene': 'pseudogene',
            'nonsense_mediated_decay': 'nmd',
            'polymorphic_pseudogene': 'pseudogene',
            'transcribed_processed_pseudogene': 'pseudogene',
            'TEC': 'others',
            'sense_overlapping': 'processed_transcript',
            'TR_J_gene': 'others',
            'IG_J_gene': 'others',
            'IG_V_pseudogene': 'pseudogene',
            'TR_V_gene': 'others',
            'rRNA': 'processed_transcript',
            'TR_C_gene': 'others',
            'scaRNA': 'processed_transcript',
            'IG_V_gene': 'others',
            'pseudogene': 'pseudogene',
            'bidirectional_promoter_lncRNA': 'processed_transcript',
            'TR_V_pseudogene': 'pseudogene',
            'Mt_tRNA': 'processed_transcript',
            'unitary_pseudogene': 'pseudogene',
            'IG_C_gene': 'others',
            'IG_pseudogene': 'pseudogene',
            'transcribed_unitary_pseudogene': 'pseudogene',
            'IG_C_pseudogene': 'pseudogene',
            'IG_D_gene': 'others',
            'non_coding': 'processed_transcript',
            'ribozyme': 'processed_transcript',
            '3prime_overlapping_ncRNA': 'processed_transcript',
            'TR_J_pseudogene': 'pseudogene',
            'IG_J_pseudogene': 'pseudogene',
            'non_stop_decay': 'nmd',
            'sRNA': 'processed_transcript',
            'TR_D_gene': 'others',
            'scRNA': 'processed_transcript',
            'vaultRNA': 'processed_transcript',
            'Mt_rRNA': 'processed_transcript',
            'macro_lncRNA': 'processed_transcript',
            'lincrna': 'processed_transcript',
            'lncRNA': 'processed_transcript',
            'vault_RNA': 'processed_transcript',
    'translated_unprocessed_pseudogene': 'pseudogene'
            }

In [4]:
path_106 = 'data/other_species/human_106/human.txt'
path_95 = 'data/other_species/human_95/human.txt'
excel_path = 'data/other_species/human_95/OP_HS_Ref79_Alt81_gene-name.csv'

In [5]:
def get_trxps_info(path):
    dataset = dict()
    with open(path, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for n, row in tqdm(enumerate(reader)):
            if n==0:
                cols = row
                continue
            line = dict(zip(cols, row))
            trx = line['Transcript stable ID']
            dataset[trx] = {'gene_id': line['Gene stable ID'],
                            'prot_id': line['Protein stable ID'],
                            'gene_name': line['Gene name'],
                            'trx_biotype': biotype_grouping[line['Transcript type']],
                            'gene_biotype': biotype_grouping[line['Gene type']]}
    return dataset

In [6]:
human_106 = get_trxps_info(path_106)
human_95 = get_trxps_info(path_95)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [14]:
import pandas as pd
data_frame = pd.read_csv(excel_path, header=None)
ref_to_cryp = dict()
for index, row in tqdm(data_frame.iterrows()):
    prot_id = row[1]
    prev_biotype = ''
    new_biotype = ''
    #if prot_id.startswith('ENSP'): 
    for trx, obj in human_95.items():
        if obj['prot_id'] == prot_id:
            prev_biotype = obj['gene_biotype']
    for trx, obj in human_106.items():
        if obj['prot_id'] == prot_id:
            new_biotype = obj['gene_biotype']
    ref_to_cryp[index] = {'trx_id': row[0],
                         'prev_id': row[1],
                          'new_id': row[5],
                         'gene_name': row[2],
                         '1.6_type': row[3],
                         '2.0_type': row[4],
                         'prev_biotype': prev_biotype,
                         'new_biotype': new_biotype}

0it [00:00, ?it/s]

In [1]:
[x for x in ref_to_cryp.values() if x['prev_id'].startswith('ENSP00000378355.1')]

NameError: name 'ref_to_cryp' is not defined